***The Euler Approximation***


Below is the use of the Euler-Maruyama approximation to solve the SDE from problem 5.9: 
$$
dX_t=\ln(1+X_t^2)\,dt+1_{\{X_t>0\}}X_t dB_t,
\qquad
X_0=a\in\mathbb{R}.
$$

In [1]:
import numpy as np
#Rng is the random number generator
rng = np.random.default_rng(seed=42)
#Number of simulations
M = 10**5

In [2]:
#X_0 is the initial value
X_0 = 1 
#T is the terminal time
T = 1.0
#N is the number of time steps
N = 6*10**3

The step used for the algorithm.
$$
X_{n+1}=X_{n}+\ln(1+X_{n}^2) \Delta + (1_{\{X_n>0\}}X_n)(B_{n\Delta+1}-B_{n\Delta+1})
$$

In [3]:
#\Delta(dt) is the time step size
dt = T/N
#B is the Brownian motion increments
dB = rng.normal(0.0, np.sqrt(dt), (N,M))
X = np.zeros((N+1,M))
X[0,:] = X_0
n = 0
while n <= N-1:
    X[n+1,:] = X[n,:] + np.log(1+X[n,:]**2 )*dt + np.where(X[n,:] > 0, X[n,:], 0)*dB[n,:]
    n += 1

In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Time vector (use all time increments)
t = np.linspace(0.0, T, N + 1)

# Summary statistics at each time step
mean = X.mean(axis=1)
var  = X.var(axis=1)
std  = np.sqrt(var)

p5   = np.quantile(X, 0.05, axis=1)
p95  = np.quantile(X, 0.95, axis=1)

# ±1 std bands
m1_lo, m1_hi = mean - std, mean + std

# Randomly sample 10 paths (not closest-to-mean)
num_paths = 10
path_idx = rng.choice(M, size=num_paths, replace=False)
Xsel = X[:, path_idx]

# Density background: use ALL time increments and ALL trajectories
Tgrid = np.repeat(t, M)
Xgrid = X.ravel()

# Robust y-range
y_lo = np.quantile(Xgrid, 0.001)
y_hi = np.quantile(Xgrid, 0.999)
pad = 0.05 * (y_hi - y_lo)
y_lo -= pad
y_hi += pad

# Build 2D histogram (time bins match N)
bins_t = N
bins_y = 200
H, xedges, yedges = np.histogram2d(
    Tgrid, Xgrid,
    bins=[bins_t, bins_y],
    range=[[0.0, T], [y_lo, y_hi]]
)
H = H.T

# High-resolution figure
dpi = 350
fig, ax = plt.subplots(figsize=(14, 7), dpi=dpi)

# Show density background (log scale)
H_log = np.log1p(H)
im = ax.imshow(
    H_log, origin="lower",
    extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]],
    aspect="auto"
)

# -----------------------------
# Manual colors (distinct from background)
# -----------------------------
c_mean = "white"
c_paths = "cyan"
c_pct = "magenta"     # p5 and p95 same color
c_std = "lime"        # +/- 1 std same color

# Plot random sample paths
for k in range(num_paths):
    ax.plot(t, Xsel[:, k], linewidth=1.1, color=c_paths,
            label="10 random sample paths" if k == 0 else None)

# 5%-95% band + bounds (same color for p5 and p95)
ax.fill_between(t, p5, p95, color=c_pct, alpha=0.18, label="5th–95th percentile band")
ax.plot(t, p5,  linewidth=1.8, color=c_pct, label="5th/95th percentiles (bounds)")
ax.plot(t, p95, linewidth=1.8, color=c_pct)

# ±1 std band + bounds (same color for both bounds)
ax.plot(t, m1_hi, linewidth=1.6, color=c_std, label="Mean + 1 std (bounds)")

# Mean line (on top)
ax.plot(t, mean, linewidth=2.6, color=c_mean, label="Mean")

ax.set_xlabel("Time t")
ax.set_ylabel("X(t)")
ax.set_title("Simulated SDE Paths with Density, Percentiles, and ±1 Std Bands")

# Info box (left)
text = (
    f"N = {N:,} steps, M = {M:,} simulations, dt = {dt:.2e}\n"
    f"X_0 = {X_0}, T = {T}\n"
    f"At t=T: mean={mean[-1]:.4f}, var={var[-1]:.4f}, std={std[-1]:.4f}, "
    f"p5={p5[-1]:.4f}, p95={p95[-1]:.4f}"
)
ax.text(
    0.02, 0.98, text, transform=ax.transAxes, va="top", ha="left",
    bbox=dict(boxstyle="round", facecolor="white", alpha=0.85)
)

# Legend (upper-right) explaining each element
ax.legend(loc="upper right", framealpha=0.9)

# Colorbar for density
cbar = fig.colorbar(im, ax=ax, pad=0.015)
cbar.set_label("log(1 + density count)")

ax.set_xlim(0.0, T)
ax.set_ylim(y_lo, y_hi)

plt.tight_layout()
